In [ ]:
import stim
import numpy as np

circuit = stim.Circuit()


distance = 3
grid_size = distance + 1

def q(i, j):
    return i * grid_size + j

for i in range(distance):
    for j in range(distance):
        if (i + j) % 2 == 1:
            ancilla = q(i, j)
            data_neighbors = [q(i+1, j), q(i, j+1), q(i-1, j), q(i, j-1)]
            data_neighbors = [n for n in data_neighbors if 0 <= n < grid_size**2]

            circuit.append("H", [ancilla])
            for data in data_neighbors:
                circuit.append("CX", [ancilla, data])
            circuit.append("H", [ancilla]) 
            circuit.append("M", [ancilla]) 

for i in range(distance):
    for j in range(distance):
        if (i + j) % 2 == 0: 
            ancilla = q(i, j)
            data_neighbors = [q(i+1, j), q(i, j+1), q(i-1, j), q(i, j-1)]
            data_neighbors = [n for n in data_neighbors if 0 <= n < grid_size**2]

            for data in data_neighbors:
                circuit.append("CX", [data, ancilla])
            circuit.append("M", [ancilla])

#Inject an X error on a data qubit
#circuit.append("X_ERROR(0.1)", [q(1, 1)]) 

samples = circuit.sample(num_samples=5)
print("\n🔹 Measurement Samples:")
print(samples)

simulator = stim.TableauSimulator()
simulator.do(circuit)

print("\n🔹 Final Stabilizer Tableau:")
print(simulator.current_tableau()) 
